#### Review and tutorial of 

# Deep Neural Networks as Gaussian Processes 
Jaehoon Lee, Yasaman Bahri, Roman Novak , Samuel S. Schoenholz, Jeffrey Pennington, Jascha Sohl-Dickstein
  
#### Presented by  
[Jason Deng](mailto:dengzj@Hotmail.com)  
[Alexander Dubitskiy](mailto:ald028@g.harvard.edu)  
[Zheng Yang](mailto:zhengyang@g.harvard.edu)  
[Sean Tierney](mailto:set936@g.harvard.edu) 

#### Background information

The question of defining meaningful priors for a neural network was first addressed by Radford M. Neal in his 1994 paper [Priors for infinite networks](ftp://www.cs.toronto.edu/dist/radford/pin.pdf).  
In the paper he studied a neural network with a real valued input of size I, one hidden layer of size H with sigmoidal transfer function and real valued output of size O. This network can be described with the following equations:
$$ f_k(x) = b_k + \sum_{i=1}^{H} v_{jk} h_j(x) $$ 
$$ h_j(x) = tanh( a_j + \sum_{i=1}^{I} u_{ij} x_i) $$

The author suggested Gaussian priors over the network weights $ b_k \propto \mathcal{N}(0, \sigma_b) , v_{jk} \propto \mathcal{N}(0, \sigma_v), a_j \propto \mathcal{N}(0, \sigma_a) , u_{ij} \propto \mathcal{N}(0, \sigma_u) $ and showed that when $H \to \infty$ the prior joint distribution converges to a multivariate Gaussian with zero means and covarinace:
  
$$ E[f_k(x) f_k(x\prime)]  = \sigma_{b}^2 + \sum_{j} \sigma_{v}^2 E[h_{j}(x)h_{j}(x \prime)] = \sigma_{b}^2 + w_{v}^2 C(x, x \prime) $$
and this is a Gaussian process.

This work was further extended by Christopher K. I. Williams in his 1996 paper [Computing with infinite networks](https://papers.nips.cc/paper/1197-computing-with-infinite-networks.pdf).  
Williams showed how to calculate the corresponding kernels for single-layered neural networks with sigmoidal and Gaussian transfer functions.  
  
$$ C_{erf}(x, x \prime) = \frac{2}{\pi} \sin^{-1} \frac{2 x^{T} \Sigma x \prime }{\sqrt{(1 + 2 x^{T} \Sigma x ) (1 + 2 x \prime^{T} \Sigma x \prime )}} $$
  
$$ C_{G}(x, x \prime) = (\frac{\sigma_e}{\sigma_u})^d \exp(- \frac{x^{T}x}{2 \sigma_{m}^2}) \exp(- \frac{(x - x^{T})^{T} (x - x^{T})}{2 \sigma_{s}^2}) \exp(- \frac{x \prime^{T}x \prime}{2 \sigma_{m}^2}) $$  
where 
$$ \frac{1}{\sigma_{e}^2} = \frac{2}{\sigma_{g}^2} + \frac{1}{\sigma_{u}^2} , \sigma_{s}^2 = 2 \sigma_{g}^2 + \frac{\sigma_{g}^4}{\sigma_{u}^2} , \sigma_{m}^2 = 2 \sigma_{u}^2 + \sigma_{g}^2 $$  
and $ \sigma_{g} $ is the width of the Gaussian transfer function. 

####  Summary of the kernel computation for a multi-layer neural network.

Consider an $L$-hidden-layer fully-connected neural network with hidden layers of width $N_l$ (for layer $l$) and pointwise nonlinearities $\phi$. Let $x \in \Re^{d_{in}}$ denote the input to the network, and let $z^L \in \Re^{d_{out}}$ denote it output. The $i$th component of activations in the $l$th layer, post-nonlinearity and post-affine trnsformatin, are denoted $x_i^l$ and $z_i^l$ repectively. Weight and bias parameters for the $l$th layer have components $W_{ij}^l, b_i^l$, which are independent and randomly drawn, and we take them all to have zero mean and variances $\sigma_w^2/N_l$ and $\sigma_b^2$, repectively. $GP(\mu, K)$ denotes a Gaussian process with mean and covariance functions $\mu(\cdotp), K(\cdotp, \cdotp)$, repectively.  

Let's review the multidimentional Central Limit Theorem first.  

We assume each infdividual $X_i$ is a random vector in $\Re^k$, with mean vector $\mu  = E(X_i)$ and these random vectors are i.i.d.,  
$$\frac{1}{n}\Sigma^n_{i = 1}X_i = 
\frac{1}{n}
\begin{bmatrix}
    \Sigma^n_{i = 1}X_{i(1)}\\
    \vdots \\
    \Sigma^n_{i = 1}X_{i(k)}
\end{bmatrix}
 = \bar{X_n}$$
And therefore, 
$$\frac{1}{\sqrt{n}}\Sigma^n_{i = 1}[X_i - E(X_i)] = \frac{1}{\sqrt{n}}\Sigma^n_{i = 1}(X_i - \mu) = \sqrt{n}(\bar{X_n} - \mu).$$  

The multidimentional Central Limit Theorem states that
$$\sqrt{n}(\bar{X_n} - \mu) \sim N_k(0, \Sigma)$$  

In a multi-layer neural network, the weights and bias parameters are taken to be i.i.d., and the post-activations $x_j^l, x_{j'}^l$, are independent for $j \neq j'$. From the multidimentional Center Limit Theorem, as $N_l \rightarrow \infty$, any finite collection of $\{z_i^l(x^{\alpha=1}), \ldots, z_i^l(x^{\alpha=k})\}$ will have a joint multivariate Guassian distribution and $z_i^l \sim GP(0, K^l).$  

The covariance is 
$$K^l(x, x') \equiv E[z^l_i(x)z^l_i(x')] = \sigma^2_b + \sigma^2_wE_{z_i^{l-1}\sim GP(0, K^{l-1})}[\phi(z_i^{l-1}(x))\phi(z_i^{l-1}(x'))].$$  

Since the expectation in above equation is over the GP governing $z_i^{l-1}$, which is equivalent to integrating against the joint distribution of only $z_i^{l-1}(x)$ and $z_i^{l-1}(x')$. This joint distirbution has zero mean and covariance matrix 
$$K = 
\begin{bmatrix}
    K^{l-1}(x, x) & K^{l-1}(x, x')\\
     K^{l-1}(x', x) &  K^{l-1}(x', x') \\
\end{bmatrix}$$  

Thus, we can introduce the shorthand
$$K^l(x, x') = \sigma^2_b + \sigma^2_wF_\phi(K^{l-1}(x, x'), K^{l-1}(x, x), K^{l-1}(x', x'))$$
to emphasize the recursive relationship between $K^l$ and $K^{l-1}$ via a deterministic function F whose form depends only on the nonlinearity $\phi$. This gives an iterative series of computations which can be performed to obtain $K^L$ for the GP describing the network's final output.

For the base case $K^0$, suppose $W^0_{ij} \sim N(0, \sigma^2_x/d_{in}), b_j^0 \sim N(0, \sigma_b^2)$; we can utilize the recursion relating $K^1$ and $K^0$, where
$$K^0(x, x') = E[z_j^0(x)z_j^0(x')] = \sigma^2_b + \sigma^2_w(\frac{x\cdot x'}{d_{in}})$$

#### Summary of the NNGP implementation

In a nutshell, this notebook implemented a Gaussian process regression with a NNGP kernel to return predicted Y_test given X_test. The followings are to explain how the implementation allows the computation of the GP regression and the kernel from a reverse-engineering perspective.

First of all, the Gaussian Process regression model was based on GPflow, and it is to estimate a GP $p(y^* | x, y, x^*) \sim N(\bar{\mu},\bar{K})$ according to the following equations (ref: equaiton (8 and 9), Lee etal, V3 March, 2018):

$$
\bar{\mu} = K^L_{x^*, D} (K^L_{D, D} + \sigma^2 \mathbb{I}_n)^{-1} t 
$$
and 
$$
\bar{K}=K^L_{x^*,x^*} - K^L_{x^*,D}(K^L_{D, D} + \sigma^2 \mathbb{I}_n)^{-1} K^{L,T}_{x^*, D} 
$$

where $\bar{\mu}$ is the mean of predicted Y, $\bar{K}$ represents the variance of the prediction. The model computes the Cholesky decompositionthe of $K^L$ as part of the algorithm for finding $\bar{\mu}$ and $\bar{K}$, using the functions defined under the class GaussianProcessRegression. This implementation was explained somewhere else (Rasmussen and Williams, Gaussian processes for machine learning, 2006). 

In order to get the $K^L$ for the model, a NNGP kernel was implemented with The key function k_full(). k_full() first computes $K^l$ that is the covariance of post-activation at given pre-activation variance and correlation. It then returns a fully stacked $K^L$ over all the layers in the neural network. The implementation was based on the following euqation (ref: equaiton (4), Lee etal, V3 March, 2018):

$$
K^l(x,x') = \sigma^2_b + \sigma^2_w \mathbb{E}_{z^{l-1}_i \sim GP(0, K^{l-1})}[\phi(z^{l-1}_i (x))\phi(z^{l-1}_i (x')]
$$

where $\sigma^2_b$ and $\sigma^2_w$ are initial values of the variance, which are initialized with the kernel. The expectation term $\mathbb{E}_{z^{l-1}_i \sim GP(0, K^{l-1})}[\phi(z^{l-1}_i (x))\phi(z^{l-1}_i (x')]$, which is a Gaussian integral, was computed via numerical integration implmentation. Briefly, a numerical integration is achieved by the linear interpolation method for all pairs of training-training and training-test points for layer $l$. First, all the inputs are processed to have identical norms. This preprocessing guarantees the identical marginal variance for each datapoint. Next, a matrix F is populated, containing a lookup table for the function $F_\phi$ , in the following euqation  (ref: equaiton (5), Lee etal, V3 March, 2018):

$$
K^l(x,x') = \sigma^2_b + \sigma^2_w F_\phi (K^{l-1}(x,x'), K^{l-1}(x,x),K^{l-1}(x',x'))
$$

The the function $F_\phi$  was approximately by bilinear interpolation into the matrix F. This is repeated recursively over all the layers. The bilinear interpolation has constant cost. 
Noted, these paramters are initialzed with the kernel: n_gauss, n_var and n_corr, determining the sampling densities for the numerical integration.  


To describe the implementation of k_full() in peudocodes.

k_full (): 
    
    Normalize input to unit variance or to fixed point variance
    
    For each hidden layer l:
        q_ab = interp.interp_lin_2d(...) # numerical integration for the expectation of the prevoius layer, E(z^{l-1}) (equation 4)
        q_ab = self.weight_var * q_ab + self.bias_var #This computes the covariance of the current layer, K^l
        
    q_ab_all = parallelly_stack(q_ab)  #K^L over all the layers
    
    return q_ab_all



In addition, the following tensorboard graph captures the workflow of k_full() .
<img src="kfull.png">


#### Python Implementation

#### Results